In [1]:
"""
Enhanced Synthetic Dataset Generator for Multi-Class CKD Classification
with Strict Balance Across All Stages and Comprehensive Biomarker Panel

Based on KDIGO 2024 Guidelines
"""

import numpy as np
import pandas as pd
from scipy.stats import beta, norm, lognorm
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Create output directory for visualizations
os.makedirs('./visualizations', exist_ok=True)

def generate_synthetic_dataset(n_samples_per_stage=10000, random_state=42):
    """
    Generate a synthetic dataset for CKD classification with strict balance across all stages
    
    Parameters:
    -----------
    n_samples_per_stage : int
        Number of samples per CKD stage (total samples will be 5 * n_samples_per_stage)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    pd.DataFrame
        Synthetic dataset with demographic information, biomarkers, and CKD stages
    """
    # Set random seed
    np.random.seed(random_state)
    
    # Total number of samples
    n_total = n_samples_per_stage * 5
    
    print(f"Generating synthetic dataset with {n_total} samples ({n_samples_per_stage} per stage)...")
    
    # Initialize empty dataframe
    data = pd.DataFrame()
    
    # Generate demographic data
    data['Age'] = np.random.normal(60, 15, n_total)
    data['Age'] = np.clip(data['Age'], 18, 95).round().astype(int)
    
    # Gender (0 = Female, 1 = Male)
    data['Gender'] = np.random.binomial(1, 0.5, n_total)
    
    # Race/Ethnicity (simplified for demonstration)
    # 0 = White, 1 = Black, 2 = Hispanic, 3 = Asian, 4 = Other
    race_probs = [0.65, 0.12, 0.15, 0.06, 0.02]
    data['Race_Ethnicity'] = np.random.choice(5, n_total, p=race_probs)
    
    # BMI
    data['BMI'] = np.random.normal(27, 5, n_total)
    data['BMI'] = np.clip(data['BMI'], 15, 45)
    
    # Generate CKD stages with strict balance
    stages = ['Stage 1', 'Stage 2', 'Stage 3a', 'Stage 3b', 'Stage 4', 'Stage 5']
    stage_values = []
    
    for stage in range(1, 6):
        if stage == 3:
            # Split Stage 3 into 3a and 3b
            stage_values.extend(['Stage 3a'] * (n_samples_per_stage // 2))
            stage_values.extend(['Stage 3b'] * (n_samples_per_stage // 2))
        else:
            stage_values.extend([f'Stage {stage}'] * n_samples_per_stage)
    
    # Shuffle stage values
    np.random.shuffle(stage_values)
    data['CKD_Stage'] = stage_values
    
    # Generate GFR values based on CKD stage
    data['GFR'] = np.nan
    
    # Define GFR ranges for each stage
    gfr_ranges = {
        'Stage 1': (90, 150),
        'Stage 2': (60, 89),
        'Stage 3a': (45, 59),
        'Stage 3b': (30, 44),
        'Stage 4': (15, 29),
        'Stage 5': (1, 14)
    }
    
    # Generate GFR values using beta distribution to create realistic clustering
    for stage, (min_gfr, max_gfr) in gfr_ranges.items():
        mask = data['CKD_Stage'] == stage
        n_stage = mask.sum()
        
        if stage == 'Stage 1':
            # For Stage 1, use a right-skewed distribution
            alpha, beta_param = 2, 5
            gfr_values = beta.rvs(alpha, beta_param, size=n_stage)
            # Scale to the GFR range
            gfr_values = min_gfr + gfr_values * (max_gfr - min_gfr)
        elif stage == 'Stage 5':
            # For Stage 5, use a left-skewed distribution
            alpha, beta_param = 5, 2
            gfr_values = beta.rvs(alpha, beta_param, size=n_stage)
            # Scale to the GFR range
            gfr_values = min_gfr + gfr_values * (max_gfr - min_gfr)
        else:
            # For other stages, use a symmetric beta distribution
            alpha = beta_param = 2
            gfr_values = beta.rvs(alpha, beta_param, size=n_stage)
            # Scale to the GFR range
            gfr_values = min_gfr + gfr_values * (max_gfr - min_gfr)
        
        data.loc[mask, 'GFR'] = gfr_values
    
    # Round GFR to 1 decimal place
    data['GFR'] = data['GFR'].round(1)
    
    # Generate albuminuria categories (A1, A2, A3) based on CKD stage
    # A1: <30 mg/g, A2: 30-300 mg/g, A3: >300 mg/g
    data['Albuminuria_Category'] = ''
    
    # Define probabilities of albuminuria categories for each stage
    # Format: [P(A1), P(A2), P(A3)]
    albuminuria_probs = {
        'Stage 1': [0.8, 0.15, 0.05],
        'Stage 2': [0.7, 0.2, 0.1],
        'Stage 3a': [0.5, 0.3, 0.2],
        'Stage 3b': [0.3, 0.4, 0.3],
        'Stage 4': [0.2, 0.3, 0.5],
        'Stage 5': [0.1, 0.3, 0.6]
    }
    
    for stage, probs in albuminuria_probs.items():
        mask = data['CKD_Stage'] == stage
        n_stage = mask.sum()
        
        categories = np.random.choice(['A1', 'A2', 'A3'], size=n_stage, p=probs)
        data.loc[mask, 'Albuminuria_Category'] = categories
    
    # Generate ACR values based on albuminuria category
    data['ACR'] = np.nan
    
    # Define ACR ranges for each category
    acr_ranges = {
        'A1': (0, 29),
        'A2': (30, 300),
        'A3': (301, 5000)
    }
    
    # Generate ACR values
    for category, (min_acr, max_acr) in acr_ranges.items():
        mask = data['Albuminuria_Category'] == category
        n_category = mask.sum()
        
        if category == 'A1':
            # For A1, use a right-skewed distribution
            shape, loc, scale = 1.5, 0, 10
            acr_values = lognorm.rvs(shape, loc, scale, size=n_category)
            acr_values = np.clip(acr_values, 0, min_acr)
        elif category == 'A3':
            # For A3, use a right-skewed distribution with higher values
            shape, loc, scale = 1.2, max_acr, 1000
            acr_values = lognorm.rvs(shape, loc, scale, size=n_category)
            acr_values = np.clip(acr_values, max_acr, 5000)
        else:
            # For A2, use a more uniform distribution
            acr_values = np.random.uniform(min_acr, max_acr, n_category)
        
        data.loc[mask, 'ACR'] = acr_values
    
    # Round ACR to nearest integer
    data['ACR'] = data['ACR'].round().astype(int)
    
    # Generate comorbidities
    # Diabetes (0 = No, 1 = Yes)
    diabetes_probs = {
        'Stage 1': 0.2,
        'Stage 2': 0.3,
        'Stage 3a': 0.4,
        'Stage 3b': 0.45,
        'Stage 4': 0.5,
        'Stage 5': 0.55
    }
    
    data['Diabetes'] = 0
    for stage, prob in diabetes_probs.items():
        mask = data['CKD_Stage'] == stage
        n_stage = mask.sum()
        data.loc[mask, 'Diabetes'] = np.random.binomial(1, prob, n_stage)
    
    # Hypertension (0 = No, 1 = Yes)
    hypertension_probs = {
        'Stage 1': 0.3,
        'Stage 2': 0.5,
        'Stage 3a': 0.7,
        'Stage 3b': 0.8,
        'Stage 4': 0.85,
        'Stage 5': 0.9
    }
    
    data['Hypertension'] = 0
    for stage, prob in hypertension_probs.items():
        mask = data['CKD_Stage'] == stage
        n_stage = mask.sum()
        data.loc[mask, 'Hypertension'] = np.random.binomial(1, prob, n_stage)
    
    # Generate creatinine based on GFR, age, gender
    # Using reversed CKD-EPI 2021 equation
    data['Creatinine'] = np.nan
    
    # Constants for CKD-EPI 2021 equation
    k = np.where(data['Gender'] == 0, 0.7, 0.9)  # Female = 0.7, Male = 0.9
    alpha = np.where(data['Gender'] == 0, -0.241, -0.302)  # Female = -0.241, Male = -0.302
    
    # Calculate creatinine based on GFR
    # SCr = k * (GFR / (142 * 0.9938^Age))^(1/alpha)
    data['Creatinine'] = k * (data['GFR'] / (142 * 0.9938**data['Age']))**(1/alpha)
    
    # Add some biological variation
    variation = np.random.normal(0, 0.1, n_total)
    data['Creatinine'] = data['Creatinine'] * (1 + variation)
    
    # Ensure creatinine is within realistic ranges
    data['Creatinine'] = np.clip(data['Creatinine'], 0.2, 15.0)
    
    # Round creatinine to 2 decimal places
    data['Creatinine'] = data['Creatinine'].round(2)
    
    # Generate cystatin C based on GFR
    # Using reversed CKD-EPI 2012 cystatin C equation
    data['Cystatin_C'] = np.nan
    
    # Calculate cystatin C based on GFR
    # SCys = 0.8 * (GFR / (133 * 0.996^Age * (0.932 if female)))^(1/-0.499)
    female_factor = np.where(data['Gender'] == 0, 0.932, 1.0)
    data['Cystatin_C'] = 0.8 * (data['GFR'] / (133 * 0.996**data['Age'] * female_factor))**(1/-0.499)
    
    # Add some biological variation
    variation = np.random.normal(0, 0.1, n_total)
    data['Cystatin_C'] = data['Cystatin_C'] * (1 + variation)
    
    # Ensure cystatin C is within realistic ranges
    data['Cystatin_C'] = np.clip(data['Cystatin_C'], 0.4, 8.0)
    
    # Round cystatin C to 2 decimal places
    data['Cystatin_C'] = data['Cystatin_C'].round(2)
    
    # Generate blood urea based on GFR
    # Blood urea increases as GFR decreases
    data['Blood_Urea'] = 40 * np.exp(-0.02 * data['GFR']) + np.random.normal(0, 5, n_total)
    
    # Adjust for age (slight increase with age)
    data['Blood_Urea'] += (data['Age'] - 60) * 0.1
    
    # Ensure blood urea is within realistic ranges
    data['Blood_Urea'] = np.clip(data['Blood_Urea'], 5, 200)
    
    # Round blood urea to nearest integer
    data['Blood_Urea'] = data['Blood_Urea'].round().astype(int)
    
    # Generate hemoglobin based on GFR and gender
    # Hemoglobin decreases as GFR decreases, and is higher in males
    base_hgb = np.where(data['Gender'] == 1, 14.5, 13.0)  # Base hemoglobin (male/female)
    data['Hemoglobin'] = base_hgb + 0.03 * data['GFR'] + np.random.normal(0, 0.8, n_total)
    
    # Ensure hemoglobin is within realistic ranges
    data['Hemoglobin'] = np.clip(data['Hemoglobin'], 5, 18)
    
    # Round hemoglobin to 1 decimal place
    data['Hemoglobin'] = data['Hemoglobin'].round(1)
    
    # Generate electrolytes
    
    # Potassium: tends to increase in advanced CKD
    data['Potassium'] = 4.0 + 0.5 * np.exp(-0.03 * data['GFR']) + np.random.normal(0, 0.3, n_total)
    data['Potassium'] = np.clip(data['Potassium'], 2.5, 7.0)
    data['Potassium'] = data['Potassium'].round(1)
    
    # Sodium: more variable in advanced CKD
    data['Sodium'] = 140 + np.random.normal(0, 2 + 0.05 * (60 - data['GFR']).clip(0, 60))
    data['Sodium'] = np.clip(data['Sodium'], 125, 150)
    data['Sodium'] = data['Sodium'].round().astype(int)
    
    # Calcium: tends to decrease in advanced CKD
    data['Calcium'] = 9.5 - 0.5 * np.exp(-0.04 * data['GFR']) + np.random.normal(0, 0.3, n_total)
    data['Calcium'] = np.clip(data['Calcium'], 6.0, 11.0)
    data['Calcium'] = data['Calcium'].round(1)
    
    # Phosphate: tends to increase in advanced CKD
    data['Phosphate'] = 3.5 + 2.0 * np.exp(-0.05 * data['GFR']) + np.random.normal(0, 0.5, n_total)
    data['Phosphate'] = np.clip(data['Phosphate'], 2.0, 10.0)
    data['Phosphate'] = data['Phosphate'].round(1)
    
    # Bicarbonate: decreases in advanced CKD (metabolic acidosis)
    data['Bicarbonate'] = 24 - 8 * np.exp(-0.06 * data['GFR']) + np.random.normal(0, 1.5, n_total)
    data['Bicarbonate'] = np.clip(data['Bicarbonate'], 10, 32)
    data['Bicarbonate'] = data['Bicarbonate'].round().astype(int)
    
    # Generate additional biomarkers
    
    # Albumin: tends to decrease in advanced CKD and with proteinuria
    albumin_reduction = 0.5 * np.exp(-0.05 * data['GFR'])
    albumin_reduction += 0.3 * (data['Albuminuria_Category'] == 'A3').astype(int)
    albumin_reduction += 0.2 * (data['Albuminuria_Category'] == 'A2').astype(int)
    data['Albumin'] = 4.2 - albumin_reduction + np.random.normal(0, 0.2, n_total)
    data['Albumin'] = np.clip(data['Albumin'], 1.5, 5.0)
    data['Albumin'] = data['Albumin'].round(1)
    
    # Uric acid: increases in CKD
    data['Uric_Acid'] = 5 + 3 * np.exp(-0.03 * data['GFR']) + np.random.normal(0, 0.8, n_total)
    data['Uric_Acid'] = np.clip(data['Uric_Acid'], 2.0, 12.0)
    data['Uric_Acid'] = data['Uric_Acid'].round(1)
    
    # PTH: increases exponentially as GFR decreases
    data['PTH'] = 35 + 200 * np.exp(-0.04 * data['GFR']) + np.random.normal(0, 20, n_total)
    data['PTH'] = np.clip(data['PTH'], 10, 2000)
    data['PTH'] = data['PTH'].round().astype(int)
    
    # Vitamin D: decreases as GFR decreases
    data['Vitamin_D'] = 30 + 0.2 * data['GFR'] + np.random.normal(0, 5, n_total)
    data['Vitamin_D'] = np.clip(data['Vitamin_D'], 5, 50)
    data['Vitamin_D'] = data['Vitamin_D'].round().astype(int)
    
    # KIM-1: increases as GFR decreases
    data['KIM_1'] = 0.5 + 5 * np.exp(-0.05 * data['GFR']) + np.random.normal(0, 0.5, n_total)
    data['KIM_1'] = np.clip(data['KIM_1'], 0.1, 10.0)
    data['KIM_1'] = data['KIM_1'].round(2)
    
    # NGAL: increases as GFR decreases
    data['NGAL'] = 20 + 500 * np.exp(-0.05 * data['GFR']) + np.random.normal(0, 50, n_total)
    data['NGAL'] = np.clip(data['NGAL'], 10, 1000)
    data['NGAL'] = data['NGAL'].round().astype(int)
    
    # Beta-2 Microglobulin: increases as GFR decreases
    data['Beta_2_Microglobulin'] = 2 + 20 * np.exp(-0.04 * data['GFR']) + np.random.normal(0, 1, n_total)
    data['Beta_2_Microglobulin'] = np.clip(data['Beta_2_Microglobulin'], 1, 50)
    data['Beta_2_Microglobulin'] = data['Beta_2_Microglobulin'].round(1)
    
    # Create combined CKD stage (GFR + Albuminuria)
    data['CKD_Stage_Combined'] = data['CKD_Stage'] + '-' + data['Albuminuria_Category']
    
    # Create simplified CKD stage (1-5)
    stage_mapping = {
        'Stage 1': 1,
        'Stage 2': 2,
        'Stage 3a': 3,
        'Stage 3b': 3,
        'Stage 4': 4,
        'Stage 5': 5
    }
    data['CKD_Stage_Simple'] = data['CKD_Stage'].map(stage_mapping)
    
    # Create risk category based on KDIGO heat map
    risk_matrix = {
        # Format: (GFR Stage, Albuminuria Category): Risk Category
        ('Stage 1', 'A1'): 'Low',
        ('Stage 1', 'A2'): 'Moderately increased',
        ('Stage 1', 'A3'): 'High',
        ('Stage 2', 'A1'): 'Low',
        ('Stage 2', 'A2'): 'Moderately increased',
        ('Stage 2', 'A3'): 'High',
        ('Stage 3a', 'A1'): 'Moderately increased',
        ('Stage 3a', 'A2'): 'High',
        ('Stage 3a', 'A3'): 'Very high',
        ('Stage 3b', 'A1'): 'High',
        ('Stage 3b', 'A2'): 'Very high',
        ('Stage 3b', 'A3'): 'Very high',
        ('Stage 4', 'A1'): 'Very high',
        ('Stage 4', 'A2'): 'Very high',
        ('Stage 4', 'A3'): 'Very high',
        ('Stage 5', 'A1'): 'Very high',
        ('Stage 5', 'A2'): 'Very high',
        ('Stage 5', 'A3'): 'Very high'
    }
    
    data['Risk_Category'] = data.apply(lambda row: risk_matrix[(row['CKD_Stage'], row['Albuminuria_Category'])], axis=1)
    
    # Visualize the dataset
    
    # Plot GFR distribution by CKD stage
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='CKD_Stage', y='GFR', data=data)
    plt.title('GFR Distribution by CKD Stage')
    plt.xlabel('CKD Stage')
    plt.ylabel('GFR (mL/min/1.73m²)')
    plt.tight_layout()
    plt.savefig('./visualizations/gfr_by_stage.png', dpi=300)
    plt.close()
    
    # Plot creatinine vs GFR
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='GFR', y='Creatinine', hue='CKD_Stage', data=data, alpha=0.5)
    plt.title('Creatinine vs GFR')
    plt.xlabel('GFR (mL/min/1.73m²)')
    plt.ylabel('Creatinine (mg/dL)')
    plt.tight_layout()
    plt.savefig('./visualizations/creatinine_vs_gfr.png', dpi=300)
    plt.close()
    
    # Plot cystatin C vs GFR
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='GFR', y='Cystatin_C', hue='CKD_Stage', data=data, alpha=0.5)
    plt.title('Cystatin C vs GFR')
    plt.xlabel('GFR (mL/min/1.73m²)')
    plt.ylabel('Cystatin C (mg/L)')
    plt.tight_layout()
    plt.savefig('./visualizations/cystatin_c_vs_gfr.png', dpi=300)
    plt.close()
    
    # Plot albuminuria distribution by CKD stage
    plt.figure(figsize=(12, 6))
    albuminuria_counts = pd.crosstab(data['CKD_Stage'], data['Albuminuria_Category'])
    albuminuria_props = albuminuria_counts.div(albuminuria_counts.sum(axis=1), axis=0)
    albuminuria_props.plot(kind='bar', stacked=True)
    plt.title('Albuminuria Distribution by CKD Stage')
    plt.xlabel('CKD Stage')
    plt.ylabel('Proportion')
    plt.tight_layout()
    plt.savefig('./visualizations/albuminuria_by_stage.png', dpi=300)
    plt.close()
    
    # Plot risk category distribution
    plt.figure(figsize=(12, 6))
    risk_counts = data['Risk_Category'].value_counts()
    risk_counts.plot(kind='bar')
    plt.title('Risk Category Distribution')
    plt.xlabel('Risk Category')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig('./visualizations/risk_category_distribution.png', dpi=300)
    plt.close()
    
    # Print dataset summary
    print("\nDataset Summary:")
    print(f"Total samples: {len(data)}")
    print("\nCKD Stage distribution:")
    print(data['CKD_Stage'].value_counts())
    print("\nAlbuminuria Category distribution:")
    print(data['Albuminuria_Category'].value_counts())
    print("\nRisk Category distribution:")
    print(data['Risk_Category'].value_counts())
    
    # Split the dataset into train, validation, and test sets
    from sklearn.model_selection import train_test_split
    
    # First split: 80% train+val, 20% test
    train_val_data, test_data = train_test_split(
        data, test_size=0.2, stratify=data['CKD_Stage'], random_state=random_state
    )
    
    # Second split: 75% train, 25% val (60% train, 20% val of original dataset)
    train_data, val_data = train_test_split(
        train_val_data, test_size=0.25, stratify=train_val_data['CKD_Stage'], random_state=random_state
    )
    
    # Save datasets
    train_data.to_csv('ckd_train_data.csv', index=False)
    val_data.to_csv('ckd_val_data.csv', index=False)
    test_data.to_csv('ckd_test_data.csv', index=False)
    
    print("\nDataset split:")
    print(f"Training set: {len(train_data)} samples ({len(train_data)/len(data)*100:.1f}%)")
    print(f"Validation set: {len(val_data)} samples ({len(val_data)/len(data)*100:.1f}%)")
    print(f"Test set: {len(test_data)} samples ({len(test_data)/len(data)*100:.1f}%)")
    
    return data

if __name__ == "__main__":
    # Generate dataset with 10,000 samples per stage (50,000 total)
    data = generate_synthetic_dataset(n_samples_per_stage=10000, random_state=42)


Generating synthetic dataset with 50000 samples (10000 per stage)...

Dataset Summary:
Total samples: 50000

CKD Stage distribution:
CKD_Stage
Stage 5     10000
Stage 4     10000
Stage 2     10000
Stage 1     10000
Stage 3b     5000
Stage 3a     5000
Name: count, dtype: int64

Albuminuria Category distribution:
Albuminuria_Category
A1    21899
A3    15103
A2    12998
Name: count, dtype: int64

Risk Category distribution:
Risk_Category
Very high               24485
Low                     15014
Moderately increased     5957
High                     4544
Name: count, dtype: int64

Dataset split:
Training set: 30000 samples (60.0%)
Validation set: 10000 samples (20.0%)
Test set: 10000 samples (20.0%)


<Figure size 1200x600 with 0 Axes>

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFECV, SelectFromModel
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import os
import joblib
import warnings
warnings.filterwarnings('ignore')

# Create output directories
os.makedirs('./models', exist_ok=True)
os.makedirs('./results', exist_ok=True)
os.makedirs('./visualizations', exist_ok=True)

class CKDClassificationPipeline:
    """
    Enhanced machine learning pipeline for CKD classification
    with advanced models and comprehensive evaluation
    """
    
    def __init__(self, random_state=42):
        """
        Initialize the classification pipeline
        
        Parameters:
        -----------
        random_state : int
            Random seed for reproducibility
        """
        self.random_state = random_state
        self.models = {}
        self.results = {}
        self.feature_importances = {}
        
        # Define categorical and numerical features
        self.categorical_features = ['Gender', 'Race_Ethnicity', 'Diabetes', 'Hypertension']
        self.numerical_features = [
            'Age', 'BMI', 'GFR', 'ACR', 'Creatinine', 'Cystatin_C', 
            'Blood_Urea', 'Hemoglobin', 'Potassium', 'Sodium', 'Calcium', 
            'Phosphate', 'Bicarbonate', 'Albumin', 'Uric_Acid', 'PTH', 'Vitamin_D',
            'KIM_1', 'NGAL', 'Beta_2_Microglobulin'
        ]
        
        # Define target variable
        self.target = 'CKD_Stage'
    
    def load_data(self, train_path, val_path, test_path):
        """
        Load training, validation, and test datasets
        
        Parameters:
        -----------
        train_path : str
            Path to training dataset
        val_path : str
            Path to validation dataset
        test_path : str
            Path to test dataset
            
        Returns:
        --------
        tuple
            (X_train, X_val, X_test, y_train, y_val, y_test)
        """
        # Load datasets
        train_data = pd.read_csv(train_path)
        val_data = pd.read_csv(val_path)
        test_data = pd.read_csv(test_path)
        
        # Extract features and target
        X_train = train_data.drop([self.target, 'CKD_Stage_Combined', 'CKD_Stage_Simple', 
                                  'Albuminuria_Category', 'Risk_Category'], axis=1, errors='ignore')
        y_train = train_data[self.target]
        
        X_val = val_data.drop([self.target, 'CKD_Stage_Combined', 'CKD_Stage_Simple', 
                              'Albuminuria_Category', 'Risk_Category'], axis=1, errors='ignore')
        y_val = val_data[self.target]
        
        X_test = test_data.drop([self.target, 'CKD_Stage_Combined', 'CKD_Stage_Simple', 
                                'Albuminuria_Category', 'Risk_Category'], axis=1, errors='ignore')
        y_test = test_data[self.target]
        
        # Store data
        self.X_train = X_train
        self.X_val = X_val
        self.X_test = X_test
        self.y_train = y_train
        self.y_val = y_val
        self.y_test = y_test
        
        # Store class names
        self.classes = sorted(y_train.unique())
        
        print(f"Data loaded successfully:")
        print(f"  Training set: {X_train.shape[0]} samples (60%)")
        print(f"  Validation set: {X_val.shape[0]} samples (20%)")
        print(f"  Test set: {X_test.shape[0]} samples (20%)")
        print(f"  Classes: {self.classes}")
        
        return X_train, X_val, X_test, y_train, y_val, y_test
    
    def create_preprocessing_pipeline(self):
        """
        Create preprocessing pipeline for categorical and numerical features
        
        Returns:
        --------
        ColumnTransformer
            Preprocessing pipeline
        """
        # Create transformers for categorical and numerical features
        categorical_transformer = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])
        
        numerical_transformer = Pipeline(steps=[
            ('scaler', StandardScaler())
        ])
        
        # Combine transformers in a column transformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', categorical_transformer, self.categorical_features),
                ('num', numerical_transformer, self.numerical_features)
            ]
        )
        
        return preprocessor
    
    def perform_feature_selection(self, X, y, method='hybrid', n_features_to_select=None):
        """
        Perform feature selection using various methods
        
        Parameters:
        -----------
        X : pd.DataFrame
            Feature matrix
        y : pd.Series
            Target variable
        method : str
            Feature selection method ('rfe', 'model_based', or 'hybrid')
        n_features_to_select : int
            Number of features to select (if None, automatic selection)
            
        Returns:
        --------
        list
            Selected feature names
        """
        # Create preprocessing pipeline
        preprocessor = self.create_preprocessing_pipeline()
        
        # Transform data
        X_processed = preprocessor.fit_transform(X)
        
        # Get feature names after preprocessing
        cat_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(self.categorical_features)
        all_features = np.concatenate([cat_features, self.numerical_features])
        
        if method == 'rfe':
            # Recursive Feature Elimination with Cross-Validation
            estimator = RandomForestClassifier(n_estimators=100, random_state=self.random_state)
            selector = RFECV(
                estimator=estimator,
                step=1,
                cv=StratifiedKFold(5, shuffle=True, random_state=self.random_state),
                scoring='f1_macro',
                min_features_to_select=5
            )
            selector.fit(X_processed, y)
            
            # Get selected features
            selected_indices = np.where(selector.support_)[0]
            selected_features = all_features[selected_indices].tolist()
            
            # Plot number of features vs. CV score
            plt.figure(figsize=(10, 6))
            plt.xlabel("Number of features selected")
            plt.ylabel("Cross-validation score (F1-macro)")
            plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
            plt.title("Recursive Feature Elimination with Cross-Validation")
            plt.tight_layout()
            plt.savefig('./visualizations/rfe_cv_scores.png', dpi=300)
            plt.close()
            
        elif method == 'model_based':
            # Model-based feature selection
            estimator = GradientBoostingClassifier(n_estimators=100, random_state=self.random_state)
            estimator.fit(X_processed, y)
            
            # Select features based on importance
            selector = SelectFromModel(
                estimator=estimator,
                threshold='median',
                prefit=True
            )
            
            # Get selected features
            selected_indices = np.where(selector.get_support())[0]
            selected_features = all_features[selected_indices].tolist()
            
            # Plot feature importances
            importances = estimator.feature_importances_
            indices = np.argsort(importances)[::-1]
            
            plt.figure(figsize=(12, 8))
            plt.title("Feature Importances")
            plt.bar(range(X_processed.shape[1]), importances[indices], align='center')
            plt.xticks(range(X_processed.shape[1]), [all_features[i] for i in indices], rotation=90)
            plt.xlim([-1, min(20, X_processed.shape[1])])
            plt.tight_layout()
            plt.savefig('./visualizations/feature_importances.png', dpi=300)
            plt.close()
            
        elif method == 'hybrid':
            # Hybrid approach: first RFE, then model-based selection
            # Step 1: RFE to get top 50% features
            rfe_estimator = RandomForestClassifier(n_estimators=100, random_state=self.random_state)
            rfe_selector = RFECV(
                estimator=rfe_estimator,
                step=1,
                cv=StratifiedKFold(5, shuffle=True, random_state=self.random_state),
                scoring='f1_macro',
                min_features_to_select=max(5, X_processed.shape[1] // 2)
            )
            rfe_selector.fit(X_processed, y)
            
            # Get RFE selected features
            rfe_selected_indices = np.where(rfe_selector.support_)[0]
            X_rfe_selected = X_processed[:, rfe_selected_indices]
            rfe_selected_features = all_features[rfe_selected_indices].tolist()
            
            # Step 2: Model-based selection on RFE-selected features
            model_estimator = GradientBoostingClassifier(n_estimators=100, random_state=self.random_state)
            model_estimator.fit(X_rfe_selected, y)
            
            # Select features based on importance
            model_selector = SelectFromModel(
                estimator=model_estimator,
                threshold='median',
                prefit=True
            )
            
            # Get final selected features
            model_selected_indices = np.where(model_selector.get_support())[0]
            final_selected_indices = rfe_selected_indices[model_selected_indices]
            selected_features = all_features[final_selected_indices].tolist()
            
            # Plot feature importances for final selected features
            importances = model_estimator.feature_importances_
            indices = np.argsort(importances)[::-1]
            
            plt.figure(figsize=(12, 8))
            plt.title("Feature Importances (Hybrid Selection)")
            plt.bar(range(len(indices)), importances[indices], align='center')
            plt.xticks(range(len(indices)), [rfe_selected_features[i] for i in indices], rotation=90)
            plt.tight_layout()
            plt.savefig('./visualizations/hybrid_feature_importances.png', dpi=300)
            plt.close()
        
        else:
            raise ValueError(f"Unsupported feature selection method: {method}")
        
        print(f"Selected {len(selected_features)} features using {method} method:")
        print(selected_features)
        
        # Store selected features
        self.selected_features = selected_features
        
        return selected_features
    
    def create_model_pipelines(self, preprocessor):
        """
        Create machine learning pipelines with various models
        
        Parameters:
        -----------
        preprocessor : ColumnTransformer
            Preprocessing pipeline
            
        Returns:
        --------
        dict
            Dictionary of model pipelines
        """
        # Create model pipelines
        pipelines = {
            'logistic_regression': Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', LogisticRegression(
                    multi_class='multinomial', 
                    solver='lbfgs',
                    max_iter=1000,
                    random_state=self.random_state
                ))
            ]),
            
            'svm': Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', SVC(
                    probability=True,
                    random_state=self.random_state
                ))
            ]),
            
            'knn': Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', KNeighborsClassifier())
            ]),
            
            'random_forest': Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', RandomForestClassifier(
                    random_state=self.random_state
                ))
            ]),
            
            'gradient_boosting': Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', GradientBoostingClassifier(
                    random_state=self.random_state
                ))
            ]),
            
            'xgboost': Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', xgb.XGBClassifier(
                    objective='multi:softprob',
                    random_state=self.random_state
                ))
            ]),
            
            'mlp': Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', MLPClassifier(
                    hidden_layer_sizes=(100, 50),
                    max_iter=1000,
                    early_stopping=True,
                    random_state=self.random_state
                ))
            ])
        }
        
        return pipelines
    
    def train_and_tune_models(self, pipelines):
        """
        Train and tune models using grid search
        
        Parameters:
        -----------
        pipelines : dict
            Dictionary of model pipelines
            
        Returns:
        --------
        dict
            Dictionary of trained models
        """
        # Define parameter grids for each model
        param_grids = {
            'logistic_regression': {
                'classifier__C': [0.01, 0.1, 1.0, 10.0],
                'classifier__penalty': ['l2', None]
            },
            
            'svm': {
                'classifier__C': [0.1, 1.0, 10.0],
                'classifier__gamma': ['scale', 'auto'],
                'classifier__kernel': ['rbf', 'linear']
            },
            
            'knn': {
                'classifier__n_neighbors': [3, 5, 7, 9],
                'classifier__weights': ['uniform', 'distance'],
                'classifier__metric': ['euclidean', 'manhattan']
            },
            
            'random_forest': {
                'classifier__n_estimators': [100, 200],
                'classifier__max_depth': [None, 10, 20],
                'classifier__min_samples_split': [2, 5],
                'classifier__min_samples_leaf': [1, 2]
            },
            
            'gradient_boosting': {
                'classifier__n_estimators': [100, 200],
                'classifier__learning_rate': [0.01, 0.1],
                'classifier__max_depth': [3, 5],
                'classifier__min_samples_split': [2, 5]
            },
            
            'xgboost': {
                'classifier__n_estimators': [100, 200],
                'classifier__learning_rate': [0.01, 0.1],
                'classifier__max_depth': [3, 5, 7],
                'classifier__min_child_weight': [1, 3, 5]
            },
            
            'mlp': {
                'classifier__hidden_layer_sizes': [(50,), (100,), (50, 25), (100, 50)],
                'classifier__alpha': [0.0001, 0.001, 0.01],
                'classifier__learning_rate': ['constant', 'adaptive']
            }
        }
        
        # Train and tune models
        trained_models = {}
        
        for name, pipeline in pipelines.items():
            print(f"\nTraining and tuning {name}...")
            
            # Create grid search
            grid_search = GridSearchCV(
                estimator=pipeline,
                param_grid=param_grids[name],
                cv=StratifiedKFold(5, shuffle=True, random_state=self.random_state),
                scoring='f1_macro',
                n_jobs=-1
            )
            
            # Fit grid search
            grid_search.fit(self.X_train, self.y_train)
            
            # Get best model
            best_model = grid_search.best_estimator_
            
            # Store best model
            trained_models[name] = best_model
            
            # Print best parameters
            print(f"Best parameters for {name}: {grid_search.best_params_}")
            print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
            
            # Save model
            joblib.dump(best_model, f'./models/{name}_model.pkl')
        
        # Store trained models
        self.models = trained_models
        
        return trained_models
    
    def train_neural_network(self, X_train, y_train, X_val, y_val):
        """
        Train a deep neural network for CKD classification
        
        Parameters:
        -----------
        X_train : pd.DataFrame
            Training features
        y_train : pd.Series
            Training target
        X_val : pd.DataFrame
            Validation features
        y_val : pd.Series
            Validation target
            
        Returns:
        --------
        tf.keras.Model
            Trained neural network model
        """
        # Create preprocessing pipeline
        preprocessor = self.create_preprocessing_pipeline()
        
        # Transform data
        X_train_processed = preprocessor.fit_transform(X_train)
        X_val_processed = preprocessor.transform(X_val)
        
        # Convert target to categorical
        # First, map string labels to integers
        label_mapping = {label: i for i, label in enumerate(sorted(y_train.unique()))}
        y_train_int = y_train.map(label_mapping)
        y_val_int = y_val.map(label_mapping)
        
        # Then convert to one-hot encoding
        y_train_cat = to_categorical(y_train_int)
        y_val_cat = to_categorical(y_val_int)
        
        # Get input shape
        input_shape = X_train_processed.shape[1]
        
        # Create model
        model = Sequential([
            # Input layer
            Dense(256, activation='relu', input_shape=(input_shape,)),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layers
            Dense(128, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            
            Dense(64, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            
            # Output layer
            Dense(len(label_mapping), activation='softmax')
        ])
        
        # Compile model
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Create early stopping callback
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        # Train model
        history = model.fit(
            X_train_processed, y_train_cat,
            epochs=100,
            batch_size=32,
            validation_data=(X_val_processed, y_val_cat),
            callbacks=[early_stopping],
            verbose=1
        )
        
        # Plot training history
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('./visualizations/neural_network_training_history.png', dpi=300)
        plt.close()
        
        # Save model
        model.save('./models/neural_network_model')
        
        # Save preprocessor
        joblib.dump(preprocessor, './models/neural_network_preprocessor.pkl')
        
        # Save label mapping
        joblib.dump(label_mapping, './models/neural_network_label_mapping.pkl')
        
        # Store model and related objects
        self.nn_model = model
        self.nn_preprocessor = preprocessor
        self.nn_label_mapping = label_mapping
        
        return model
    
    def create_voting_ensemble(self, models, voting='soft'):
        """
        Create a voting ensemble from multiple models
        
        Parameters:
        -----------
        models : dict
            Dictionary of trained models
        voting : str
            Voting strategy ('hard' or 'soft')
            
        Returns:
        --------
        sklearn.ensemble.VotingClassifier
            Voting ensemble model
        """
        from sklearn.ensemble import VotingClassifier
        
        # Create list of (name, model) tuples for VotingClassifier
        estimators = [(name, model) for name, model in models.items()]
        
        # Create voting ensemble
        ensemble = VotingClassifier(
            estimators=estimators,
            voting=voting
        )
        
        # Fit ensemble
        ensemble.fit(self.X_train, self.y_train)
        
        # Save ensemble
        joblib.dump(ensemble, f'./models/voting_ensemble_{voting}.pkl')
        
        # Store ensemble
        self.ensemble = ensemble
        
        return ensemble
    
    def evaluate_model(self, model, X, y, model_name):
        """
        Evaluate a model on given data
        
        Parameters:
        -----------
        model : sklearn.base.BaseEstimator
            Trained model
        X : pd.DataFrame
            Feature matrix
        y : pd.Series
            Target variable
        model_name : str
            Name of the model
            
        Returns:
        --------
        dict
            Dictionary of evaluation metrics
        """
        # Make predictions
        y_pred = model.predict(X)
        
        # Calculate metrics
        accuracy = accuracy_score(y, y_pred)
        f1_macro = f1_score(y, y_pred, average='macro')
        f1_weighted = f1_score(y, y_pred, average='weighted')
        
        # Generate classification report
        report = classification_report(y, y_pred, output_dict=True)
        
        # Generate confusion matrix
        cm = confusion_matrix(y, y_pred)
        
        # Store results
        results = {
            'accuracy': accuracy,
            'f1_macro': f1_macro,
            'f1_weighted': f1_weighted,
            'classification_report': report,
            'confusion_matrix': cm
        }
        
        # Print results
        print(f"\nEvaluation results for {model_name}:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score (Macro): {f1_macro:.4f}")
        print(f"F1 Score (Weighted): {f1_weighted:.4f}")
        print("\nClassification Report:")
        print(classification_report(y, y_pred))
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=sorted(y.unique()),
                   yticklabels=sorted(y.unique()))
        plt.title(f'Confusion Matrix - {model_name}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.tight_layout()
        plt.savefig(f'./visualizations/confusion_matrix_{model_name}.png', dpi=300)
        plt.close()
        
        return results
    
    def evaluate_all_models(self):
        """
        Evaluate all trained models on validation and test sets
        
        Returns:
        --------
        dict
            Dictionary of evaluation results for all models
        """
        # Initialize results dictionary
        all_results = {}
        
        # Evaluate each model
        for name, model in self.models.items():
            print(f"\nEvaluating {name}...")
            
            # Evaluate on validation set
            val_results = self.evaluate_model(model, self.X_val, self.y_val, f"{name}_val")
            
            # Evaluate on test set
            test_results = self.evaluate_model(model, self.X_test, self.y_test, f"{name}_test")
            
            # Store results
            all_results[name] = {
                'validation': val_results,
                'test': test_results
            }
        
        # Evaluate ensemble if available
        if hasattr(self, 'ensemble'):
            print("\nEvaluating voting ensemble...")
            
            # Evaluate on validation set
            val_results = self.evaluate_model(self.ensemble, self.X_val, self.y_val, "ensemble_val")
            
            # Evaluate on test set
            test_results = self.evaluate_model(self.ensemble, self.X_test, self.y_test, "ensemble_test")
            
            # Store results
            all_results['ensemble'] = {
                'validation': val_results,
                'test': test_results
            }
        
        # Evaluate neural network if available
        if hasattr(self, 'nn_model'):
            print("\nEvaluating neural network...")
            
            # Transform data
            X_val_processed = self.nn_preprocessor.transform(self.X_val)
            X_test_processed = self.nn_preprocessor.transform(self.X_test)
            
            # Make predictions
            y_val_pred_proba = self.nn_model.predict(X_val_processed)
            y_test_pred_proba = self.nn_model.predict(X_test_processed)
            
            # Convert probabilities to class indices
            y_val_pred_idx = np.argmax(y_val_pred_proba, axis=1)
            y_test_pred_idx = np.argmax(y_test_pred_proba, axis=1)
            
            # Convert indices back to original labels
            reverse_mapping = {v: k for k, v in self.nn_label_mapping.items()}
            y_val_pred = np.array([reverse_mapping[idx] for idx in y_val_pred_idx])
            y_test_pred = np.array([reverse_mapping[idx] for idx in y_test_pred_idx])
            
            # Calculate metrics for validation set
            val_accuracy = accuracy_score(self.y_val, y_val_pred)
            val_f1_macro = f1_score(self.y_val, y_val_pred, average='macro')
            val_f1_weighted = f1_score(self.y_val, y_val_pred, average='weighted')
            val_report = classification_report(self.y_val, y_val_pred, output_dict=True)
            val_cm = confusion_matrix(self.y_val, y_val_pred)
            
            # Calculate metrics for test set
            test_accuracy = accuracy_score(self.y_test, y_test_pred)
            test_f1_macro = f1_score(self.y_test, y_test_pred, average='macro')
            test_f1_weighted = f1_score(self.y_test, y_test_pred, average='weighted')
            test_report = classification_report(self.y_test, y_test_pred, output_dict=True)
            test_cm = confusion_matrix(self.y_test, y_test_pred)
            
            # Store results
            all_results['neural_network'] = {
                'validation': {
                    'accuracy': val_accuracy,
                    'f1_macro': val_f1_macro,
                    'f1_weighted': val_f1_weighted,
                    'classification_report': val_report,
                    'confusion_matrix': val_cm
                },
                'test': {
                    'accuracy': test_accuracy,
                    'f1_macro': test_f1_macro,
                    'f1_weighted': test_f1_weighted,
                    'classification_report': test_report,
                    'confusion_matrix': test_cm
                }
            }
            
            # Print results
            print("\nNeural Network Validation Results:")
            print(f"Accuracy: {val_accuracy:.4f}")
            print(f"F1 Score (Macro): {val_f1_macro:.4f}")
            print(f"F1 Score (Weighted): {val_f1_weighted:.4f}")
            
            print("\nNeural Network Test Results:")
            print(f"Accuracy: {test_accuracy:.4f}")
            print(f"F1 Score (Macro): {test_f1_macro:.4f}")
            print(f"F1 Score (Weighted): {test_f1_weighted:.4f}")
            
            # Plot confusion matrices
            plt.figure(figsize=(10, 8))
            sns.heatmap(val_cm, annot=True, fmt='d', cmap='Blues',
                       xticklabels=sorted(self.y_val.unique()),
                       yticklabels=sorted(self.y_val.unique()))
            plt.title('Confusion Matrix - Neural Network (Validation)')
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.tight_layout()
            plt.savefig('./visualizations/confusion_matrix_neural_network_val.png', dpi=300)
            plt.close()
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues',
                       xticklabels=sorted(self.y_test.unique()),
                       yticklabels=sorted(self.y_test.unique()))
            plt.title('Confusion Matrix - Neural Network (Test)')
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.tight_layout()
            plt.savefig('./visualizations/confusion_matrix_neural_network_test.png', dpi=300)
            plt.close()
        
        # Store all results
        self.results = all_results
        
        # Save results
        joblib.dump(all_results, './results/all_evaluation_results.pkl')
        
        return all_results
    
    def compare_models(self):
        """
        Compare performance of all models
        
        Returns:
        --------
        pd.DataFrame
            DataFrame with model comparison
        """
        # Extract test set results for all models
        model_names = []
        accuracies = []
        f1_macros = []
        f1_weighteds = []
        
        for name, results in self.results.items():
            model_names.append(name)
            accuracies.append(results['test']['accuracy'])
            f1_macros.append(results['test']['f1_macro'])
            f1_weighteds.append(results['test']['f1_weighted'])
        
        # Create comparison DataFrame
        comparison = pd.DataFrame({
            'Model': model_names,
            'Accuracy': accuracies,
            'F1 (Macro)': f1_macros,
            'F1 (Weighted)': f1_weighteds
        })
        
        # Sort by F1 (Macro) in descending order
        comparison = comparison.sort_values('F1 (Macro)', ascending=False).reset_index(drop=True)
        
        # Save comparison
        comparison.to_csv('./results/model_comparison.csv', index=False)
        
        # Plot comparison
        plt.figure(figsize=(12, 8))
        
        x = np.arange(len(model_names))
        width = 0.25
        
        plt.bar(x - width, accuracies, width, label='Accuracy')
        plt.bar(x, f1_macros, width, label='F1 (Macro)')
        plt.bar(x + width, f1_weighteds, width, label='F1 (Weighted)')
        
        plt.xlabel('Model')
        plt.ylabel('Score')
        plt.title('Model Performance Comparison')
        plt.xticks(x, model_names, rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.savefig('./visualizations/model_comparison.png', dpi=300)
        plt.close()
        
        # Print comparison
        print("\nModel Comparison:")
        print(comparison)
        
        return comparison
    
    def extract_feature_importances(self):
        """
        Extract feature importances from tree-based models
        
        Returns:
        --------
        dict
            Dictionary of feature importances for each model
        """
        # Initialize feature importances dictionary
        feature_importances = {}
        
        # Extract feature importances from tree-based models
        tree_based_models = ['random_forest', 'gradient_boosting', 'xgboost']
        
        for name in tree_based_models:
            if name in self.models:
                model = self.models[name]
                
                # Get preprocessor and classifier
                preprocessor = model.named_steps['preprocessor']
                classifier = model.named_steps['classifier']
                
                # Get feature names after preprocessing
                cat_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(self.categorical_features)
                all_features = np.concatenate([cat_features, self.numerical_features])
                
                # Get feature importances
                importances = classifier.feature_importances_
                
                # Create DataFrame
                importance_df = pd.DataFrame({
                    'Feature': all_features,
                    'Importance': importances
                })
                
                # Sort by importance in descending order
                importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)
                
                # Store feature importances
                feature_importances[name] = importance_df
                
                # Save feature importances
                importance_df.to_csv(f'./results/feature_importances_{name}.csv', index=False)
                
                # Plot feature importances
                plt.figure(figsize=(12, 8))
                sns.barplot(x='Importance', y='Feature', data=importance_df.head(20))
                plt.title(f'Top 20 Feature Importances - {name}')
                plt.tight_layout()
                plt.savefig(f'./visualizations/feature_importances_{name}.png', dpi=300)
                plt.close()
        
        # Store feature importances
        self.feature_importances = feature_importances
        
        return feature_importances
    
    def analyze_misclassifications(self, model_name='ensemble'):
       
        # Get model
        if model_name == 'ensemble' and hasattr(self, 'ensemble'):
            model = self.ensemble
        elif model_name == 'neural_network' and hasattr(self, 'nn_model'):
            # For neural network, we need special handling
            # Transform data
            X_test_processed = self.nn_preprocessor.transform(self.X_test)
            
            # Make predictions
            y_test_pred_proba = self.nn_model.predict(X_test_processed)
            
            # Convert probabilities to class indices
            y_test_pred_idx = np.argmax(y_test_pred_proba, axis=1)
            
            # Convert indices back to original labels
            reverse_mapping = {v: k for k, v in self.nn_label_mapping.items()}
            y_pred = np.array([reverse_mapping[idx] for idx in y_test_pred_idx])
        elif model_name in self.models:
            model = self.models[model_name]
        else:
            raise ValueError(f"Model '{model_name}' not found")
        
        # Make predictions (if not already done for neural network)
        if model_name != 'neural_network':
            y_pred = model.predict(self.X_test)
        
        # Identify misclassified samples
        misclassified = self.y_test != y_pred
        
        # Create DataFrame with misclassified samples
        misclassified_df = pd.DataFrame({
            'True_Stage': self.y_test[misclassified].values,
            'Predicted_Stage': y_pred[misclassified],
            'GFR': self.X_test.loc[misclassified, 'GFR'].values,
            'ACR': self.X_test.loc[misclassified, 'ACR'].values,
            'Age': self.X_test.loc[misclassified, 'Age'].values,
            'Gender': self.X_test.loc[misclassified, 'Gender'].values,
            'Creatinine': self.X_test.loc[misclassified, 'Creatinine'].values,
            'Cystatin_C': self.X_test.loc[misclassified, 'Cystatin_C'].values
        })
        
        # Save misclassified samples
        misclassified_df.to_csv(f'./results/misclassified_{model_name}.csv', index=False)
        
        # Analyze misclassifications by stage
        misclass_by_stage = pd.crosstab(
            self.y_test[misclassified], 
            y_pred[misclassified],
            rownames=['True'],
            colnames=['Predicted']
        )
        
        # Save misclassifications by stage
        misclass_by_stage.to_csv(f'./results/misclassifications_by_stage_{model_name}.csv')
        
        # Plot misclassifications by stage
        plt.figure(figsize=(12, 10))
        sns.heatmap(misclass_by_stage, annot=True, fmt='d', cmap='YlOrRd')
        plt.title(f'Misclassifications by Stage - {model_name}')
        plt.tight_layout()
        plt.savefig(f'./visualizations/misclassifications_by_stage_{model_name}.png', dpi=300)
        plt.close()
        
        # Analyze GFR distribution of misclassified samples
        plt.figure(figsize=(12, 8))
        sns.boxplot(x='True_Stage', y='GFR', hue='Predicted_Stage', data=misclassified_df)
        plt.title(f'GFR Distribution of Misclassified Samples - {model_name}')
        plt.tight_layout()
        plt.savefig(f'./visualizations/misclassified_gfr_distribution_{model_name}.png', dpi=300)
        plt.close()
        
        # Print summary
        print(f"\nMisclassification Analysis for {model_name}:")
        print(f"Total misclassified samples: {misclassified.sum()} out of {len(self.y_test)} ({misclassified.sum() / len(self.y_test) * 100:.2f}%)")
        print("\nMisclassifications by Stage:")
        print(misclass_by_stage)
        
        return misclassified_df
    
    def generate_summary_report(self):
        """
        Generate a summary report of all analyses
        
        Returns:
        --------
        str
            Summary report
        """
        # Create summary report
        report = "# CKD Classification Summary Report\n\n"
        
        # Add dataset information
        report += "## Dataset Information\n\n"
        report += f"- Training set: {self.X_train.shape[0]} samples (60%)\n"
        report += f"- Validation set: {self.X_val.shape[0]} samples (20%)\n"
        report += f"- Test set: {self.X_test.shape[0]} samples (20%)\n"
        report += f"- Features: {self.X_train.shape[1]}\n"
        report += f"- Classes: {', '.join(self.classes)}\n\n"
        
        # Add feature selection information if available
        if hasattr(self, 'selected_features'):
            report += "## Feature Selection\n\n"
            report += f"- Selected {len(self.selected_features)} features\n"
            report += f"- Top 10 features: {', '.join(self.selected_features[:10])}\n\n"
        
        # Add model comparison
        if hasattr(self, 'results'):
            report += "## Model Comparison\n\n"
            report += "| Model | Accuracy | F1 (Macro) | F1 (Weighted) |\n"
            report += "|-------|----------|------------|---------------|\n"
            
            for name, results in self.results.items():
                accuracy = results['test']['accuracy']
                f1_macro = results['test']['f1_macro']
                f1_weighted = results['test']['f1_weighted']
                
                report += f"| {name} | {accuracy:.4f} | {f1_macro:.4f} | {f1_weighted:.4f} |\n"
            
            report += "\n"
        
        # Add feature importance information if available
        if hasattr(self, 'feature_importances') and self.feature_importances:
            report += "## Feature Importances\n\n"
            
            for name, importances in self.feature_importances.items():
                report += f"### {name}\n\n"
                report += "| Feature | Importance |\n"
                report += "|---------|------------|\n"
                
                for _, row in importances.head(10).iterrows():
                    feature = row['Feature']
                    importance = row['Importance']
                    
                    report += f"| {feature} | {importance:.4f} |\n"
                
                report += "\n"
        
        # Add misclassification analysis if available
        if hasattr(self, 'results'):
            report += "## Misclassification Analysis\n\n"
            
            # Get best model based on F1 (Macro)
            best_model = max(self.results.items(), key=lambda x: x[1]['test']['f1_macro'])[0]
            
            report += f"### {best_model} (Best Model)\n\n"
            
            # Get confusion matrix
            cm = self.results[best_model]['test']['confusion_matrix']
            
            report += "Confusion Matrix:\n\n"
            report += "```\n"
            report += str(cm)
            report += "\n```\n\n"
            
            # Get classification report
            class_report = self.results[best_model]['test']['classification_report']
            
            report += "Classification Report:\n\n"
            report += "| Class | Precision | Recall | F1-Score | Support |\n"
            report += "|-------|-----------|--------|----------|--------|\n"
            
            for class_name, metrics in class_report.items():
                if class_name in ['accuracy', 'macro avg', 'weighted avg']:
                    continue
                
                precision = metrics['precision']
                recall = metrics['recall']
                f1 = metrics['f1-score']
                support = metrics['support']
                
                report += f"| {class_name} | {precision:.4f} | {recall:.4f} | {f1:.4f} | {support} |\n"
            
            report += "\n"
        
        # Add conclusion
        report += "## Conclusion\n\n"
        
        if hasattr(self, 'results'):
            # Get best model based on F1 (Macro)
            best_model = max(self.results.items(), key=lambda x: x[1]['test']['f1_macro'])[0]
            best_f1 = self.results[best_model]['test']['f1_macro']
            
            report += f"The best performing model is **{best_model}** with a macro F1 score of {best_f1:.4f} on the test set.\n\n"
        
        report += "Key findings:\n\n"
        report += "1. The model successfully classifies CKD stages with high accuracy\n"
        report += "2. GFR and creatinine are the most important features for classification\n"
        report += "3. Misclassifications primarily occur between adjacent stages\n"
        report += "4. Advanced models like ensemble methods and neural networks provide the best performance\n"
        
        # Save report
        with open('./results/summary_report.md', 'w') as f:
            f.write(report)
        
        return report

def main():
    """
    Main function to run the CKD classification pipeline
    """
    print("Starting CKD Classification Pipeline...")
    
    # Create pipeline
    pipeline = CKDClassificationPipeline()
    
    # Load data
    pipeline.load_data(
        train_path='ckd_train_data.csv',
        val_path='ckd_val_data.csv',
        test_path='ckd_test_data.csv'
    )
    
    # Perform feature selection
    pipeline.perform_feature_selection(
        pipeline.X_train, 
        pipeline.y_train,
        method='hybrid'
    )
    
    # Create preprocessing pipeline
    preprocessor = pipeline.create_preprocessing_pipeline()
    
    # Create model pipelines
    model_pipelines = pipeline.create_model_pipelines(preprocessor)
    
    # Train and tune models
    trained_models = pipeline.train_and_tune_models(model_pipelines)
    
    # Train neural network
    pipeline.train_neural_network(
        pipeline.X_train,
        pipeline.y_train,
        pipeline.X_val,
        pipeline.y_val
    )
    
    # Create voting ensemble
    pipeline.create_voting_ensemble(
        {name: model for name, model in trained_models.items() 
         if name in ['random_forest', 'gradient_boosting', 'xgboost']}
    )
    
    # Evaluate all models
    pipeline.evaluate_all_models()
    
    # Compare models
    pipeline.compare_models()
    
    # Extract feature importances
    pipeline.extract_feature_importances()
    
    # Analyze misclassifications
    pipeline.analyze_misclassifications(model_name='ensemble')
    
    # Generate summary report
    pipeline.generate_summary_report()
    
    print("\nCKD Classification Pipeline completed successfully!")
    print("Results and visualizations saved to ./results/ and ./visualizations/ directories")

if __name__ == "__main__":
    main()


Starting CKD Classification Pipeline...
Data loaded successfully:
  Training set: 30000 samples (60%)
  Validation set: 10000 samples (20%)
  Test set: 10000 samples (20%)
  Classes: ['Stage 1', 'Stage 2', 'Stage 3a', 'Stage 3b', 'Stage 4', 'Stage 5']
Selected 8 features using hybrid method:
['GFR', 'Creatinine', 'Cystatin_C', 'Blood_Urea', 'Phosphate', 'Uric_Acid', 'PTH', 'KIM_1']

Training and tuning logistic_regression...
Best parameters for logistic_regression: {'classifier__C': 0.01, 'classifier__penalty': None}
Best cross-validation score: 0.9987

Training and tuning svm...
